In [ ]:
import pystablemotifs as sm
import pyboolnet
import pystablemotifs.export as ex
import networkx as nx
from timeit import default_timer

In [ ]:
rules_d='''

dhs-16 *= daf-2
daf-9 *= not dhs-16 or ncr or daf-12
daf-12 *= not daf-9 or not daf-7
daf-16 *= not akt or daf-12
pdk-1 *= age-1
age-1 *= aap-1
aap-1 *= daf-2
akt *= pdk-1 
daf-2 *= (ins-7 and daf-28) or (not ins-1 or not ins-18)
dauer *= daf-16 and daf-12 
ins-7 *= daf-7 or not daf-16 
daf-7 *= (tax-4 and cmk-1) and not hsf-1
daf-11 *= not srbc 
daf-28 *= cmk-1 
daf-5 *= daf-3 
daf-3 *= not daf-8-14
daf-8-14 *= daf-1-4
daf-1-4 *= daf-7
ins-1 *= daf-5
hsf-1 *= not daf-2
srbc *= pher
tax-4 *= daf-11
ins-18 *= daf-16
'''


In [ ]:
primes_d = sm.format.create_primes(rules_d)
sm.format.pretty_print_prime_rules(primes_d)


In [ ]:
#analyzing the network
ar2 = sm.AttractorRepertoire.from_primes(primes_d, max_simulate_size=50)

In [ ]:
ar2.summary()

In [ ]:
#finding unique stable motifs
ar2.succession_diagram.get_motifs()

In [ ]:
#computing driver sets - using stable motif histories that result in the set target, 
#identifies internal driver nodes and lists them as control sets

start=default_timer()
interventions = ar2.succession_diagram.reprogram_to_trap_spaces(logically_fixed=target,target_method='history',
                                               driver_method='internal')
end=default_timer()
print("Run Time:",end-start) 
print("Sets found:")
for x in interventions: print({k:v for k,v in sorted(x.items())})


In [ ]:
# Determine which parts of the control set drive which stable motifs by comparing the overlap
# Note that this can also be done "by eye" by comparing the control set with the succession diagram node or edge labels for small systems
target_state={'hsf-1': 0, 'daf-7': 1, 'ins-7': 1, 'daf-2': 1}
motifs = ar.succession_diagram.get_motifs()
control_sets = ar.succession_diagram.reprogram_to_trap_spaces(logically_fixed=target_state,target_method='history',driver_method='internal')

print(f'Displaying which control nodes are used to drive individual stable motifs in each control set for attaining {target_state}:\n')
for control_set in control_sets:
    ldoi,_=sm.drivers.logical_domain_of_influence(control_set,primes) # determine which nodes are driven by this particular control set
    for motif in motifs:
        motif_drivers = motif.items() & control_set.items() # which control set nodes might drive the stable motif    
        is_driver=sm.drivers.fixed_implies_implicant(ldoi,motif) # verify that the control set really does drive the stable motif
    
        # display
        if len(motif_drivers) > 0 and is_driver:
            print(f'{control_set=},{motif=},{motif_drivers=}')
    print()
    
print("--------")
# Get a list of all possible internal driver nodes for each stable motif in the succession diagram 
# (this might be slow for large networks with many/big stable motifs)
print("Displaying which internal driver node sets exist for all stable motifs:")
for motif in motifs:
    print(f'all internal driver node sets for {motif}:')
    for s in sm.drivers.internal_drivers(motif,primes):
        print(f'\t{s}')

In [9]:
# define the target state and compute the control sets
#target_state = {'daf-3': 0, 'daf-8-14': 1, 'hsf-1': 0, 'daf-7': 1, 'ins-1': 0, 'daf-5': 0, 'daf-1-4': 1, 'daf-2': 1}
#target_state= {'aap-1': 0, 'age-1': 0, 'akt': 0, 'cmk-1': 0, 'daf-1-4': 0, 'daf-11': 0, 'daf-12': 1, 'daf-16': 1, 'daf-2': 0, 'daf-28': 0, 'daf-3': 1, 'daf-5': 1, 'daf-7': 0, 'daf-8-14': 0, 'daf-9': 1, 'dauer': 1, 'dhs-16': 0, 'hsf-1': 1, 'ins-1': 1, 'ins-18': 1, 'ins-7': 0, 'ncr': 0, 'pdk-1': 0, 'pher': 1, 'srbc': 1, 'tax-4': 0}

target_state={'aap-1': 1, 'age-1': 1, 'akt': 1, 'cmk-1': 1, 'daf-1-4': 1, 'daf-11': 1, 'daf-12': 0, 'daf-16': 0, 'daf-2': 1, 'daf-28': 1, 'daf-3': 0, 'daf-5': 0, 'daf-7': 1, 'daf-8-14': 1, 'daf-9': 1, 'dauer': 0, 'dhs-16': 1, 'hsf-1': 0, 'ins-1': 0, 'ins-18': 0, 'ins-7': 1, 'ncr': 1, 'pdk-1': 1, 'pher': 0, 'srbc': 0, 'tax-4': 1}

In [11]:
control_sets = ar2.succession_diagram.reprogram_to_trap_spaces(logically_fixed=target_state,target_method='history',driver_method='internal')

print("control sets are:", control_sets)
print("____")
# get a list of stable motifs that appear in the succession diagram 
# (note that some of these are only be stable motifs in a system that has been reduced by substituting "true" stable motifs)
motifs = ar2.succession_diagram.get_motifs()
print("stable motifs are:", motifs)


control sets are: [{'ncr': 1, 'cmk-1': 1, 'pher': 0, 'ins-7': 1}, {'ncr': 1, 'cmk-1': 1, 'pher': 0, 'hsf-1': 0}, {'ncr': 1, 'cmk-1': 1, 'pher': 0, 'daf-7': 1}, {'ncr': 1, 'cmk-1': 1, 'pher': 0, 'daf-2': 1}, {'ncr': 1, 'cmk-1': 1, 'pher': 0, 'daf-5': 0}, {'ncr': 1, 'cmk-1': 1, 'pher': 0, 'ins-1': 0}, {'ncr': 1, 'cmk-1': 1, 'pher': 0, 'daf-8-14': 1}, {'ncr': 1, 'cmk-1': 1, 'pher': 0, 'daf-3': 0}, {'ncr': 1, 'cmk-1': 1, 'pher': 0, 'daf-1-4': 1}, {'ncr': 1, 'cmk-1': 1, 'pher': 0, 'ins-18': 0}, {'ncr': 1, 'cmk-1': 1, 'pher': 0, 'daf-16': 0}, {'ncr': 1, 'cmk-1': 1, 'pher': 0, 'aap-1': 1, 'daf-12': 0}, {'ncr': 1, 'cmk-1': 1, 'pher': 0, 'daf-12': 0, 'pdk-1': 1}, {'ncr': 1, 'cmk-1': 1, 'pher': 0, 'daf-12': 0, 'age-1': 1}, {'ncr': 1, 'cmk-1': 1, 'pher': 0, 'daf-12': 0, 'akt': 1}]
____
stable motifs are: [{'daf-3': 1, 'ins-18': 1, 'daf-2': 0, 'daf-16': 1, 'daf-5': 1, 'daf-12': 1, 'ins-7': 0, 'daf-7': 0, 'ins-1': 1, 'hsf-1': 1, 'daf-1-4': 0, 'daf-8-14': 0}, {'daf-2': 1, 'hsf-1': 0, 'daf-3': 0, 'da

In [13]:
# Determine which parts of the control set drive which stable motifs by comparing the overlap
# Note that this can also be done "by eye" by comparing the control set with the succession diagram node or edge labels for small systems
print('Displaying which control nodes are used to drive individual stable motifs in each control set for attaining {target_state}:\n')
for control_set in control_sets:
    ldoi,_=sm.drivers.logical_domain_of_influence(control_set,primes_d) # determine which nodes are driven by this particular control set
    for motif in motifs:
        motif_drivers = motif.items() & control_set.items() # which control set nodes might drive the stable motif    
        is_driver=sm.drivers.fixed_implies_implicant(ldoi,motif) # verify that the control set really does drive the stable motif
    
        # display
        if len(motif_drivers) > 0 and is_driver:
            print(f'{control_set=},{motif=},{motif_drivers=}')
    print()
    
print("--------")

Displaying which control nodes are used to drive individual stable motifs in each control set for attaining {target_state}:

control_set={'ncr': 1, 'cmk-1': 1, 'pher': 0, 'ins-7': 1},motif={'cmk-1': 1, 'pher': 0, 'ncr': 1},motif_drivers={('cmk-1', 1), ('pher', 0), ('ncr', 1)}
control_set={'ncr': 1, 'cmk-1': 1, 'pher': 0, 'ins-7': 1},motif={'hsf-1': 0, 'ins-7': 1, 'daf-2': 1, 'daf-7': 1},motif_drivers={('ins-7', 1)}

control_set={'ncr': 1, 'cmk-1': 1, 'pher': 0, 'hsf-1': 0},motif={'daf-2': 1, 'hsf-1': 0, 'daf-3': 0, 'daf-7': 1, 'daf-1-4': 1, 'daf-8-14': 1, 'ins-1': 0, 'daf-5': 0},motif_drivers={('hsf-1', 0)}
control_set={'ncr': 1, 'cmk-1': 1, 'pher': 0, 'hsf-1': 0},motif={'daf-12': 0, 'daf-2': 1, 'hsf-1': 0, 'pdk-1': 1, 'ins-18': 0, 'daf-7': 1, 'aap-1': 1, 'akt': 1, 'age-1': 1, 'daf-16': 0},motif_drivers={('hsf-1', 0)}
control_set={'ncr': 1, 'cmk-1': 1, 'pher': 0, 'hsf-1': 0},motif={'cmk-1': 1, 'pher': 0, 'ncr': 1},motif_drivers={('cmk-1', 1), ('pher', 0), ('ncr', 1)}
control_set={'ncr'